In [ ]:
# energy generation market in Israel
# Data from Noga -  https://www.noga-iso.co.il/

import pandas as pd
import os

DATADIR = '../data/noga'

In [ ]:
import os
import pandas as pd

def read_xlsx_files_concat(DATADIR):
    dataframes = []
    column_names = None

    # Iterate over files in the directory
    for file in os.listdir(DATADIR):
        if file.endswith(".xlsx"):
            file_path = os.path.join(DATADIR, file)
            # Read the Excel file into a dataframe, skipping the first row
            df = pd.read_excel(file_path, skiprows=1)

            # Append the dataframe to the list
            dataframes.append(df)

    # Concatenate the dataframes into a single dataframe
    concatenated_df = pd.concat(dataframes, ignore_index=True)

    return concatenated_df

In [ ]:
df = read_xlsx_files_concat(DATADIR)

df.rename(columns={'תאריך' : 'date', 'שעה' : 'time'}, inplace=True)
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%d/%m/%Y %H:%M:%S')
df = df.set_index('datetime').drop(['date', 'time'], axis=1)
df_bak = df.copy()

In [ ]:
df.columns

In [ ]:
df = df_bak.copy().resample('1D').mean()

In [ ]:
cols = ['ייצור משקי בפועל', 'ניהול הביקוש', 'סהכ מתחדשות', 'גז טבעי', 'פחם']
plotme = df.drop(columns=cols).fillna(0)

In [ ]:
import plotly.express as px

fig = px.bar(df, x=plotme.index, y=plotme.columns, barmode='stack')
#fig = px.line(df, x=plotme.index, y=plotme.columns)
fig.show()

In [ ]:
plotme = df[cols].fillna(0)

In [ ]:
import plotly.express as px

fig = px.bar(df, x=plotme.index, y=plotme.columns, barmode='stack')
#fig = px.line(df, x=plotme.index, y=plotme.columns)
fig.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

col = 'פוטו וולאטי'
rcol = col[::-1]

# Filter the dataframe for the last 14 days
df_filtered = df_bak.copy().resample('1H').mean()
df_filtered = df_filtered[df_filtered.index >= df_filtered.index[-1] - pd.DateOffset(days=14)]
df_filtered['hour'] = df_filtered.index.strftime('%H:%M')

# Plot the line chart with error bars
sns.lineplot(data=df_filtered, x='hour', y=col, errorbar='ci')
plt.xlabel('hour')
plt.ylabel(rcol)
plt.title(f'value of {rcol} conditioned on hour of the day')
plt.show()


In [ ]:
dst = pd.to_datetime('2023-03-24')

In [ ]:
df2 = df_bak.copy().resample('30T').mean()
df2 = df2[df2.index >= dst]
df2['hour'] = df2.index.strftime('%H:%M')

In [ ]:
df3 = pd.DataFrame(
    index = df2.query('hour == "12:30"').index,
    data={
    '12' : df2.query('hour == "12:30"')[col].values,
    '13' : df2.query('hour == "13:30"')[col].values,
    '14' : df2.query('hour == "14:30"')[col].values,
})
df3['12minus13'] = df3['12'] - df3['13']
df3['14minus13'] = df3['14'] - df3['13']

In [ ]:
import plotly.express as px

# Assuming you have a DataFrame named 'df2' with columns '12minus13' and '14minus13'

fig = px.line(df3, x=df3.index, y=['12minus13', '14minus13'])

#fig.show()

In [ ]:
from scipy import stats

# Perform paired t-test
t_statistic, p_value = stats.ttest_rel(df3['14'], df3['13'])

# Check if the p-value is less than the significance level (e.g., 0.05)
alpha = 0.05
if p_value < alpha:
    print("Reject null hypothesis. Column '12' values are significantly higher than column '13'.")
else:
    print("Fail to reject null hypothesis. No significant difference between column '12' and column '13'.")
p_value